GroupBy function

In [24]:
from pyspark.sql import SparkSession

In [25]:

spark=SparkSession.builder.appName('Agg').getOrCreate()


In [26]:
spark

In [27]:
df_pyspark=spark.read.csv('C:/Users/rammo/OneDrive/Desktop/Book2.csv',header=True,inferSchema=True)

In [28]:
df_pyspark.show()

+----+---------+------------+------+
| _c0|     Name| Departments|salary|
+----+---------+------------+------+
|NULL|    Krish|Data Science| 10000|
|NULL|    Krish|         IOT|  5000|
|NULL|   Mahesh|    Big Data|  4000|
|NULL|    Krish|    Big Data|  4000|
|NULL|   Mahesh|Data Science|  3000|
|NULL|Sudhanshu|Data Science| 20000|
|NULL|Sudhanshu|         IOT| 10000|
|NULL|Sudhanshu|    Big Data|  5000|
|NULL|    Sunny|Data Science| 10000|
|NULL|    Sunny|    Big Data|  2000|
+----+---------+------------+------+



In [29]:
## Groupby
### Grouped to find the maximum salary
df_pyspark.groupBy('Name').sum().show()

+---------+-----------+
|     Name|sum(salary)|
+---------+-----------+
|Sudhanshu|      35000|
|    Sunny|      12000|
|    Krish|      19000|
|   Mahesh|       7000|
+---------+-----------+



In [30]:
df_pyspark.groupBy('Name').avg().show()

+---------+------------------+
|     Name|       avg(salary)|
+---------+------------------+
|Sudhanshu|11666.666666666666|
|    Sunny|            6000.0|
|    Krish| 6333.333333333333|
|   Mahesh|            3500.0|
+---------+------------------+



In [31]:
### Groupby Departmernts  which gives maximum salary
df_pyspark.groupBy('Departments').sum().show()

+------------+-----------+
| Departments|sum(salary)|
+------------+-----------+
|         IOT|      15000|
|    Big Data|      15000|
|Data Science|      43000|
+------------+-----------+



In [32]:
df_pyspark.groupBy('Departments').count().show()

+------------+-----+
| Departments|count|
+------------+-----+
|         IOT|    2|
|    Big Data|    4|
|Data Science|    4|
+------------+-----+



agg Method: 
The agg method is used to perform aggregate operations on one or more columns of the DataFrame.
aggregate operations

In [33]:
df_pyspark.agg({'Salary':'sum'}).show()

+-----------+
|sum(Salary)|
+-----------+
|      73000|
+-----------+



Example of pyspark ML

VectorAssembler is an essential tool in PySpark for converting multiple feature columns into a single vector column required for most machine learning algorithms. It simplifies data preparation by combining features and can be used in conjunction with other feature transformers and scalers for advanced data processing tasks.

In [43]:
df_pyspark=spark.read.option('header','true').csv('C:/Users/rammo/OneDrive/Desktop/booknew.csv',inferSchema=True)
df_pyspark.show()
#if csv file has null values it will get error.

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|     John| 32|         5| 20000|
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



VectorAssembler:-
VectorAssembler is a feature transformer provided by Apache Spark's MLlib (Machine Learning Library). It is used to combine multiple columns of a DataFrame into a single vector column, which is essential for many machine learning algorithms and operations.

Prepares Data for ML Algorithms:-
 Many Spark MLlib algorithms expect a single vector column for features, making VectorAssembler a crucial step in the machine learning pipeline.

In [44]:
from pyspark.ml.feature import VectorAssembler

featureassembler = VectorAssembler(inputCols=["age","Experience"],
                                 outputCol="Independent Features")

In [45]:
output=featureassembler.transform(df_pyspark)

In [49]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|     John| 32|         5| 20000|          [32.0,5.0]|
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [58]:
finalized_data=output.select("Independent Features","Salary")
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [32.0,5.0]| 20000|
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [57]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data= finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [59]:
### Coefficients
regressor.coefficients

DenseVector([-246.8845, 1855.2824])

In [53]:
### Intercepts
regressor.intercept

19847.812338296568

In [60]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [61]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [24.0,3.0]| 20000|18792.635006174947|
|          [30.0,8.0]| 25000|26587.739979791182|
+--------------------+------+------------------+



In [62]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1397.5524868081175, 1989324.2358707364)